In [2]:
import os

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\office\\vc'

In [1]:
from trainer import Trainer, TrainerArgs
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

In [6]:
from pathlib import Path
# or for interactive environments:
output_path = Path.cwd()

dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=output_path / "artifacts/data_ingestion/LJSpeech-1.1"
)
audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)


In [ ]:
config = VitsConfig(
    audio=audio_config,
    run_name="vits_ljspeech",
    batch_size=32,
    eval_batch_size=16,
    batch_group_size=5,
    num_loader_workers=8,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    text_cleaner="english_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
)

In [8]:
ap = AudioProcessor.init_from_config(config)
tokenizer, config = TTSTokenizer.init_from_config(config)


 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [9]:
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)


 | > Found 829 files in D:\office\vc\artifacts\data_ingestion\LJSpeech-1.1


In [10]:
model = Vits(config, ap, tokenizer, speaker_manager=None)


In [11]:
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 12
 | > Num. of Torch Threads: 6
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=d:\office\vc\vits_ljspeech-May-22-2025_09+58AM-f4cb597
d:\miniconda\envs\vc\lib\site-packages\trainer\trainer.py:552: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()

 > Model has 83059180 parameters


In [21]:
trainer.fit()



 > EPOCH: 0/1000
 --> d:\office\vc\vits_ljspeech-May-21-2025_08+11PM-f4cb597


[*] Pre-computing phonemes...


  0%|          | 3/821 [00:02<10:57,  1.24it/s]

hi ɪsnt æz bɪɡ bʌt hz stɪl kwaɪt pɑpjəlɚ ɪv hɚd ðə seɪm θɪŋ əbaʊt hɪz kɑntɛnt nɛvɚ wɔt͡ʃt hɪm mʌt͡ʃ
 [!] Character '͡' not found in the vocabulary. Discarding it.


  5%|▌         | 42/821 [00:15<04:41,  2.77it/s]
 > Keyboard interrupt detected.
 > Saving model before exiting...

 > CHECKPOINT : d:\office\vc\vits_ljspeech-May-21-2025_08+11PM-f4cb597\checkpoint_0.pth
 ! Run is kept in d:\office\vc\vits_ljspeech-May-21-2025_08+11PM-f4cb597


: 